# TODO: Create Features for neighbour edges (H-N) and nodes (N)

### Currently we use neighbour (H) as simplified features for (H-N)
### We need to prepare the dataframes of features below:

In [2]:
import pandas as pd

In [3]:
locationexposure = pd.read_csv('rural_resurgence/locationExposure/DeWitt_Texas_UnitedStates.csv')
# how to collate this into the probability of spread? TODO: which one to use for countyname?


In [4]:
#social_score
unacast = pd.read_csv('rural_resurgence/unacast/unacast.csv')
unacast['n_grade'] = unacast['totalGrade'].astype('category').cat.codes
unacast['social_score'] = unacast['n_grade']/unacast['n_grade'].max()

In [5]:


l_h2n = [[0.3,0.7], [0.2,0.6], [0.1,0.5], [0.4,0.4], [0.5,0.8]]

h2n = pd.DataFrame(l_h2n, index=['Greenwood County','Elbert County','McCormick County','Anderson County','Lafayette Parish'], columns=['covid_rate','social_score'])

l_N = [[0.3,0.8,0.4], [0.3,0.8,0.4], [0.3,0.8,0.4]]

N = pd.DataFrame(l_N, index=['Abbeville County','Acadia Parish','Greenwood County'], columns=['medical_preparedness','county_health','Social_status'])


# Probability formulas for P(H-N)  and P(N)

#### Starting with the simplistic view of multiplying likelihood

In [8]:
h2n['ph2n'] = h2n['covid_rate'] * h2n['social_score'] ##tune here
N['pN'] = N['medical_preparedness'] * N['county_health'] * N['Social_status'] ##tune here

# Defining the list of neighbours 
### TODO: We need to create a dictionary of neighbours for each county below:

In [9]:
l_neighbours = {'Buffalo':['Hand','Hyde','Lyman'], 'Hand':['Faulk','Hyde','Buffalo'], 'Hyde':['Faulk','Hand','Buffalo']}
neighbours = pd.DataFrame.from_dict(l_neighbours)


In [10]:
# Code for extracting list of neighbours
mile25 = pd.read_csv('data/sf12010countydistance25miles.csv', encoding = "ISO-8859-1")
neighbours = mile25[['countyname1','countyname2']].pivot(columns='countyname1',values='countyname2').dropna(how='all')
neighbour_dict = mile25.groupby('countyname1')['countyname2'].apply(list).to_dict()

# Collecting the P(H-N) probabilites of neighbours spreading covid

In [24]:
h2n_dict = h2n.to_dict()
neighboursprob = neighbours.copy()
for county in neighbours.columns:
    newcol = county + 'ph2n'
    neighboursprob[newcol] = neighbours[county].map(h2n_dict['ph2n'])

In [26]:
neighboursprob[['Abbeville County','Abbeville Countyph2n']].dropna()

countyname1,Abbeville County,Abbeville Countyph2n
4933,Greenwood County,0.21
4934,Elbert County,0.12
4935,McCormick County,0.05
4936,Anderson County,0.16


# Probability of a county getting covid
#### P(n) = P(N) * P(H-N)
#### P(N) = sum(P(N.medical_preparedness), P(N.county_health), P(N.social_status))

In [21]:
pn = {}
for county in neighbours.columns:
    countyph2n = county + "ph2n"
    ph2nsum = neighboursprob[countyph2n].sum() ##tune here
    print(county)
    pn[county] = ph2nsum * N.at[county,"pN"] ##tune here
    break
    

Abbeville County


# We sort pn and find the counties with the highest probabilites
### E.g. If we used features for Feb 2020, these probabilites would be the prediction for Mar 2020

In [22]:
pd.DataFrame.from_dict(pn, orient='index').sort_values(0, ascending=False)

,0
Abbeville County,0.05184


# TODO: We need to get some ground truth to compare with

### The above is for example features from Feb 2020, we will need to compare the output with Mar 2020 counties covid count